In [1]:
%matplotlib inline
import numpy as np
np.random.seed(2016)
import matplotlib.pyplot as plt

import os
import glob
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from scipy.misc import imread
from scipy.misc import imresize

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import __version__ as keras_version

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
DATA_PATH = '/a/data/fisheries_monitoring/data/'

In [3]:
def load_cropped_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NoF']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join(DATA_PATH + 'classifiers/superbox/', fld, '*.jpg')
        files = sorted(glob.glob(path))
        for fl in files:
            flbase = os.path.basename(fl)
            img = image.load_img(fl, target_size=(224, 224))
            img = image.img_to_array(img)
            X_train.append(img)
            X_train_id.append(fld + '/' + flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

def read_and_normalize_cropped_train_data():
    train_data, train_target, train_id = load_cropped_train()

    print('Convert to numpy...')
    train_data = np.array(train_data)
    train_target = np.array(train_target)

    print('Convert to float...')
    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 8)

    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id

In [4]:
train_data, train_target, train_id = read_and_normalize_cropped_train_data()

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder OTHER (Index: 4)
Load folder SHARK (Index: 5)
Load folder YFT (Index: 6)
Load folder NoF (Index: 7)
Read train data time: 28.93 seconds
Convert to numpy...
Convert to float...
('Train shape:', (3777, 224, 224, 3))
(3777, 'train samples')


In [ ]:
n = np.random.choice(train_data.shape[0])
img = train_data[n]
print train_id[n]
_, ax = plt.subplots(1, 1, figsize=(60, 9))
ax.imshow(img)

ALB/img_01620.jpg


In [28]:
train_id[n]

'NoF/img_03908.jpg'

In [5]:
base_model = ResNet50(weights='imagenet', include_top = False, input_shape=(224,224,3))

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(8, activation='softmax')(x)
model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='sgd', loss='categorical_crossentropy')

In [6]:
batch_size = 16
nb_epoch = 30
random_state = 51

X_train, X_test, y_train, y_test = train_test_split(train_data, train_target, test_size=0.2, random_state=0)

In [7]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),]
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True,
          verbose=2, validation_data=(X_test, y_test), callbacks=callbacks)
model.save(DATA_PATH + 'models/ResNet50_SuperBox.h5')

Train on 3021 samples, validate on 756 samples
Epoch 1/30
46s - loss: 1.0205 - val_loss: 2.0390
Epoch 2/30
45s - loss: 0.6130 - val_loss: 2.3426
Epoch 3/30
45s - loss: 0.4810 - val_loss: 2.3537
Epoch 4/30
45s - loss: 0.3858 - val_loss: 2.0463
Epoch 5/30
45s - loss: 0.3247 - val_loss: 1.0338
Epoch 6/30
45s - loss: 0.2892 - val_loss: 0.3947
Epoch 7/30
45s - loss: 0.2568 - val_loss: 0.2845
Epoch 8/30
45s - loss: 0.2197 - val_loss: 0.3789
Epoch 9/30
45s - loss: 0.2036 - val_loss: 0.2638
Epoch 10/30
45s - loss: 0.1834 - val_loss: 0.2673
Epoch 11/30
45s - loss: 0.1569 - val_loss: 0.2782
Epoch 12/30
45s - loss: 0.1523 - val_loss: 0.2376
Epoch 13/30
45s - loss: 0.1365 - val_loss: 0.2359
Epoch 14/30
45s - loss: 0.1205 - val_loss: 0.2360
Epoch 15/30
45s - loss: 0.1196 - val_loss: 0.2453
Epoch 16/30
45s - loss: 0.1031 - val_loss: 0.2323
Epoch 17/30
45s - loss: 0.1009 - val_loss: 0.2284
Epoch 18/30
45s - loss: 0.0898 - val_loss: 0.2292
Epoch 19/30
45s - loss: 0.0731 - val_loss: 0.2531
Epoch 20/30


In [8]:
predictions_valid = model.predict(X_test.astype('float32'), batch_size=batch_size, verbose=2)
score = log_loss(y_test, predictions_valid)

In [9]:
print "log loss score: ", score

log loss score:  0.218303046877


In [10]:
from sklearn.metrics import accuracy_score
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(predictions_valid, axis = 1)
acc = accuracy_score(y_true, y_pred, normalize=True, sample_weight=None)

In [11]:
print "accuracy: ", acc

accuracy:  0.937830687831


In [13]:
error_true = []
error_pred = []
for n in xrange(len(y_true)):
    if y_true[n] != y_pred[n]:
        error_true.append(y_true[n])
        error_pred.append(y_pred[n])
print "true: ", error_true[0:35]
print "pred: ", error_pred[0:35]
print "true: ", error_true[35:]
print "pred: ", error_pred[35:]

true:  [6, 0, 6, 0, 0, 2, 6, 6, 1, 4, 6, 0, 5, 0, 6, 2, 6, 6, 6, 0, 0, 6, 0, 5, 0, 6, 1, 6, 5, 0, 0, 6, 2, 0, 6]
pred:  [4, 6, 0, 4, 4, 0, 0, 0, 0, 6, 0, 4, 0, 6, 0, 0, 0, 0, 0, 7, 6, 1, 6, 6, 1, 0, 6, 0, 4, 5, 4, 1, 1, 4, 0]
true:  [6, 0, 6, 0, 1, 4, 2, 6, 6, 0, 0, 6]
pred:  [0, 5, 1, 6, 0, 0, 6, 1, 0, 6, 4, 2]
